In [27]:
'''
This notebook is used for hyperparameter search for the rater CNN.
It utilizes the Hyperopt library, as well as Hyperas, a wrapper for it to make defining 
the search space more readable.

Hyperopt: https://github.com/hyperopt/hyperopt
Hyperas:  https://github.com/maxpumperla/hyperas
''' 

'\nThis notebook is used for hyperparameter search for the rater CNN.\nIt utilizes the Hyperopt library, as well as Hyperas, a wrapper for it to make defining \nthe search space more readable.\n\nHyperopt: https://github.com/hyperopt/hyperopt\nHyperas:  https://github.com/maxpumperla/hyperas\n'

In [28]:
from hyperopt import Trials, STATUS_OK, tpe

from keras.engine import  Model
from keras.layers import Flatten, Dense, Input, Dropout
from keras.optimizers import Adam
from keras.preprocessing import image
from keras import backend as K
from keras_vggface.vggface import VGGFace
from keras_vggface import utils
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras import regularizers
from keras.models import model_from_json

import os
import time
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

from hyperas import optim
from hyperas.distributions import choice, uniform, conditional

In [29]:
#cfg = K.tf.ConfigProto()
#cfg.gpu_options.allow_growth = True
#K.set_session(K.tf.Session(config=cfg))

In [30]:
def limit_mem():
    K.get_session().close()
    cfg = K.tf.ConfigProto()
    cfg.gpu_options.allow_growth = True
    K.set_session(K.tf.Session(config=cfg))

In [31]:
print(K.tensorflow_backend._get_available_gpus())

['/job:localhost/replica:0/task:0/device:GPU:0']


In [32]:
def data():
    """
    Loads and prepares data.
    Done in a separate function to load it only once, instead of at every evaluation.
    """
    
    #define paths
    BASE_DIR = '../project/all_females' #image folder
    RATING_DIR = './Projekt_SGE_Assessment_ErikK.txt' #text file with user ratings
    
    N_IMAGES = len(os.listdir(BASE_DIR))
    IMAGE_SIZE = (200, 200)

    X_train = np.zeros((N_IMAGES, IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

    for idx, _im in enumerate(sorted(os.listdir(BASE_DIR))):
        # Change the image path with yours.
        _img = image.load_img(os.path.join(BASE_DIR, _im), target_size=IMAGE_SIZE)
        _x = image.img_to_array(_img)
        _x = np.expand_dims(_x, axis=0)
        X_train[idx, :, :, :] = utils.preprocess_input(_x, version=1) / .255 
    
    assert ~np.any(np.isnan(X_train))
    
    # Load ratings
    ratings = np.genfromtxt(RATING_DIR)
    ratings = to_categorical(ratings, num_classes=6)
    
    # Standardize ratings
    ratings_z = (ratings - np.mean(ratings)) / np.std(ratings)
    
    x_train, x_test, y_train, y_test = train_test_split(X_train, ratings_z, test_size=0.2, random_state=42)
    
    return x_train, y_train, x_test, y_test


In [33]:
def create_model(x_train, y_train, x_test, y_test):
    '''
    Model to be optimized.
    {{}} signify parameters which will be optimized.
    
    Returns:
        loss - Evaluation metric to be optimized.
        status - hyperopt.STATUS_OK because model always yields valid (albeit possibly bad) results
        model - created model
    '''
    #K.clear_session()
    #del history
    #del model
    #gc.collect()
    #gc.collect()
    #gc.collect()
    #time.sleep(1)
    
    #limit_mem()
    
    #K.clear_session()
    #tf.reset_default_graph()    

    input_size = 200
    vgg_base = VGGFace(include_top=False, input_shape=(input_size, input_size, 3), pooling='max')
    
    # Add custom layers
    #last_layer = vgg_base.get_layer('global_max_pooling2d_1').output
    last_layer = vgg_base.layers[-1].output
    X = Dropout({{uniform(0, 1)}})(last_layer)
    
    X = Dense(256, kernel_regularizer=regularizers.l2({{uniform(0, 1)}}), activation={{choice(['relu', 'sigmoid'])}}, name='fc6')(X)
    X = Dropout({{uniform(0, 1)}})(X)
    
    X = Dense(128, kernel_regularizer=regularizers.l2({{uniform(0, 1)}}), activation={{choice(['relu', 'sigmoid'])}}, name='fc7')(X)
    X = Dropout({{uniform(0, 1)}})(X)
    
    if conditional({{choice(['two', 'three'])}}) == 'three':
        #add another dense layer
        X = Dense(128, kernel_regularizer=regularizers.l2({{uniform(0, 1)}}), activation={{choice(['relu', 'sigmoid'])}}, name='fc8')(X)
        X = Dropout({{uniform(0, 1)}})(X)
    
    output = Dense(6, kernel_regularizer=regularizers.l2({{uniform(0, 1)}}), activation='softmax')(X)
    
    model = Model(inputs=vgg_base.input, outputs=output)
    
    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all convolutional InceptionV3 layers
    for layer in vgg_base.layers:
        layer.trainable = False

    opt = Adam(lr={{uniform(0, 1)}},clipnorm=1.0)

    model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, ratings_z, batch_size=32, epochs=50, verbose=2, validation_data=(x_test, y_test))
    score, acc = model.evaluate(x_test, y_test, verbose=0)
    
    #save architecture as json for more memory efficiency
    json_string = model.to_json()
    
    #del history
    #del model
    #gc.collect()
    #gc.collect()
    #gc.collect()
    #time.sleep(1)    
    
    
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': (json_string,opt)}

In [34]:
#vgg_base = VGGFace(include_top=False, input_shape=(200, 200, 3), pooling='max')
#vgg_base.summary()

In [35]:
#run the hyperparameter search
best_run, best_params = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=30,
                                          trials=Trials(),
                                          notebook_name='hyperSearch')

>>> Imports:
#coding=utf-8

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.engine import Model
except:
    pass

try:
    from keras.layers import Flatten, Dense, Input, Dropout
except:
    pass

try:
    from keras.optimizers import Adam
except:
    pass

try:
    from keras.preprocessing import image
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    from keras_vggface.vggface import VGGFace
except:
    pass

try:
    from keras_vggface import utils
except:
    pass

try:
    from keras.utils import to_categorical
except:
    pass

try:
    from keras.callbacks import ModelCheckpoint
except:
    pass

try:
    from keras import regularizers
except:
    pass

try:
    from keras.models import model_from_json
except:
    pass

try:
    import os
except:
    pass

try:
    import time
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from sklearn.model_selection import train_test_split
e

Test accuracy: 0.0789473684864086
Train on 568 samples, validate on 114 samples
Epoch 1/3
 - 8s - loss: 841.6241 - acc: 0.2905 - val_loss: 167.9681 - val_acc: 0.2719
Epoch 2/3
 - 7s - loss: 86.8045 - acc: 0.3134 - val_loss: 28.0369 - val_acc: 0.3421
Epoch 3/3
 - 7s - loss: 23.5878 - acc: 0.3063 - val_loss: 12.2074 - val_acc: 0.2719
Test accuracy: 0.27192982534567517
Train on 568 samples, validate on 114 samples
Epoch 1/3
 - 9s - loss: 190.9213 - acc: 0.2993 - val_loss: 185.0769 - val_acc: 0.3421
Epoch 2/3
 - 8s - loss: 172.9346 - acc: 0.3222 - val_loss: 259.7233 - val_acc: 0.3421
Epoch 3/3
 - 8s - loss: 200.6317 - acc: 0.3292 - val_loss: 198.1981 - val_acc: 0.3421
Test accuracy: 0.3421052673406768
Train on 568 samples, validate on 114 samples
Epoch 1/3
 - 10s - loss: 3861.1825 - acc: 0.3081 - val_loss: 1530.5921 - val_acc: 0.3421
Epoch 2/3
 - 8s - loss: 728.3574 - acc: 0.3028 - val_loss: 814.4466 - val_acc: 0.3421
Epoch 3/3
 - 8s - loss: 478.4507 - acc: 0.2905 - val_loss: 533.5249 - va

In [36]:
#recreate model from best_params
K.clear_session()
sess = tf.Session()
K.set_session(sess)
#tf.reset_default_graph()
best_archit, best_optim = best_params
model = model_from_json(best_archit)
model.compile(loss='categorical_crossentropy', optimizer=best_optim)

data_train, label_train, data_test, label_test = data()
#print(data_test.shape)
print("Evalutation of best performing model:")
print(model.evaluate(data_test, label_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

Evalutation of best performing model:
114/114 [==============================] - 1s 13ms/step
205.21312164842035
Best performing model chosen hyper-parameters:
{'Dropout': 0.8469072279547732, 'Dropout_1': 0.3210389307864787, 'Dropout_2': 0.5948712568443267, 'Dropout_3': 0.4970559482092457, 'Dropout_4': 0.06765709934504838, 'Dropout_5': 0.9662681038993752, 'Dropout_6': 0.011106434718081704, 'Dropout_7': 0.9770005173795487, 'Dropout_8': 0.8366666847115819, 'activation': 0, 'activation_1': 1, 'activation_2': 0, 'conditional': 0}


In [37]:
#model.save('opt_love_classifier_0.h5')

#throws error: ValueError: Cannot use the 
#given session to evaluate tensor: the tensor's graph 
#is different from the session's graph.

#couldn't resolve this, so model is saved as JSON + weight file instead:

In [38]:
#save model
with open("opt_love_classifier_0.json", "w") as json_file:
    json_file.write(best_archit)
model.save_weights("opt_love_classifier_0.h5")

In [39]:
#reload model
json_file = open('opt_love_classifier_0.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#load weights into new model
loaded_model.load_weights("opt_love_classifier_0.h5")
loaded_model.compile(loss='categorical_crossentropy', optimizer=best_optim)
print(model.evaluate(data_test, label_test))

114/114 [==============================] - 1s 11ms/step
205.21312164842035
